# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("../Resources/output_data.csv")
cities_pd.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ridgecrest,35.6225,-117.6709,59.00,38,40,20.71,US,1613118396
1,Östersund,63.1792,14.6357,19.40,86,100,17.27,SE,1613118484
2,Bengkulu,-3.8004,102.2655,85.42,65,19,9.44,ID,1613118485
3,Dikson,73.5069,80.5464,-17.77,89,100,8.48,RU,1613118485
4,Atuona,-9.8000,-139.0333,78.82,77,1,11.03,PF,1613118485


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_pd[["Lat", "Lng"]].astype(float)

# Store Humidity in humidity
humidity = cities_pd["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_cities_pd = cities_pd.loc[(cities_pd["Max Temp"] >= 70) & (cities_pd["Max Temp"] <= 85) & 
                                 (cities_pd['Wind Speed'] <=10) & (cities_pd['Cloudiness'] == 0)].dropna()

narrow_cities_pd 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,East London,-33.0153,27.9116,84.20,70,0,4.61,ZA,1613118303
20,Simao,22.7886,100.9748,71.64,33,0,5.84,CN,1613118487
26,Busselton,-33.6500,115.3333,79.00,46,0,3.00,AU,1613118488
33,Arlit,18.7369,7.3853,76.26,15,0,4.52,NE,1613118489
55,Bosaso,11.2842,49.1816,77.00,69,0,9.73,SO,1613118492
92,Saldanha,-33.0117,17.9442,77.00,86,0,3.44,ZA,1613118498
181,Herat Province,34.5000,62.0000,71.60,15,0,3.44,AF,1613118350
187,Makkah al Mukarramah,21.4267,39.8261,81.57,41,0,3.53,SA,1613118511
190,Port Elizabeth,-33.9180,25.5701,77.00,83,0,9.22,ZA,1613118511
196,Nioro,13.3500,-15.7500,70.68,19,0,6.58,GM,1613118512


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = narrow_cities_pd
hotel_df
# mistakenly added airport name instead of hotel name and had to delete the column
hotel_df["Hotel Name"] = ""
#column_drop = hotel.drop(columns=['Airport Name'])
#column_drop

#hotel_df = column_drop


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in cities_pd.iterrows():
   
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row['City']
   
    params["location"] = f"{lat},{lng}"

    
    response = requests.get(base_url, params=params).json()
    results = response['results']

   
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

        

Closest hotel in Ridgecrest is Azul Inn & Suites.
Closest hotel in Östersund is Clarion Hotel Grand Östersund.
Closest hotel in Bengkulu is Mercure Bengkulu.
Missing field/result... skipping.
Closest hotel in Atuona is Hôtel Hiva Oa Hanakee Lodge.
Closest hotel in Hilo is Hilo Hawaiian Hotel.
Closest hotel in Ushuaia is Wyndham Garden Ushuaia Hotel del Glaciar.
Closest hotel in Ereymentau is Gostinitsa "Alem".
Closest hotel in Mossamedes is Hotel Chik Chik Namibe.
Missing field/result... skipping.
Closest hotel in Arkhipo-Osipovka is Roza Vetrov Novaya.
Closest hotel in Punta Arenas is Hotel Cabo de Hornos.
Closest hotel in Necochea is Hotel Ñikén Spa & Business Center.
Closest hotel in Puerto Ayora is La Isla Hotel.
Closest hotel in East London is Blue Lagoon Hotel.
Closest hotel in New Norfolk is Tynwald Willow Bend Estate.
Closest hotel in Howard Springs is buffalo beach hostel.
Closest hotel in Ostrovnoy is Sberbank Rossii.
Closest hotel in Kapaa is Sheraton Kauai Resort at Coconut

Closest hotel in Balyqshy is Marriott Executive Apartments Atyrau.
Closest hotel in Aldan is Aldan Gostinichnyy Kompleks.
Closest hotel in Narsaq is Igdlo Guesthouse.
Closest hotel in Touros is Hotel Vila Gale Touros.
Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel in Elliot is The Guardians Lodge.
Closest hotel in Mareeba is The Cape Gateway Motel.
Closest hotel in Rawson is Complejo Bahia Ballenas.
Closest hotel in Jamestown is Chautauqua Harbor Hotel.
Closest hotel in Njombe is Hillside Hotel.
Closest hotel in Clyde River is Naujaaraaluit – Clyde River Hotel.
Closest hotel in Ugoofaaru is Ungoofaaru Inn.
Closest hotel in Rapid City is Tru By Hilton Rapid City Rushmore.
Closest hotel in Pisco is HOSTAL MIRAMAR.
Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel in Anadyr is Chukotka hotel.
Closest hotel in Port Blair is J Hotel.
Closest hotel in Namatanai is Namatanai Hotel.
Closest hotel in Imbituba is Praia Hotel Imbi

Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel in Ozernovskiy is Sberbank Rossii.
Closest hotel in Opuwo is Ohakane Guesthouse.
Closest hotel in Dali is The Jade Emu International Hostel.
Closest hotel in Yaring is Mdic.
Closest hotel in Santo Antônio do Içá is Hotel Vitória Régia.
Closest hotel in Pangnirtung is Auyuittuq Lodge (Inns North).
Missing field/result... skipping.
Closest hotel in Qaqortoq is Hotel Qaqortoq.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel in Verāval is HOTEL PLATINUM.
Closest hotel in Luang Prabang is Sofitel Luang Prabang.
Closest hotel in Kalmar is Calmar Stadshotell | ProfilHotels.
Missing field/result... skipping.
Closest hotel in Mikhaylovka is гостиница Комфорт.
Closest hotel in Çubuk is Gökçe Otel.
Missing field/result... skipping.
Closest hotel in Henties Bay is Desert Rose.
Closest hotel in Los Altos Hills is Creekside Inn.
Closest hotel in Rovani

Closest hotel in Muzhi is Sberbank.
Closest hotel in Bonthe is Bonthe Holiday Village.
Closest hotel in Llaillay is HOSTAL EDWARDS 454.
Closest hotel in City of San Pedro is Palines Apartment and Guesthouse.


ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
